## random.shuffle 기능을 이용하여 
## training data 와 test data 분리 하는 DataGeneration class 이용

In [1]:
import tensorflow as tf
import numpy as np
import random
from datetime import datetime

In [2]:
class DataGeneration:
    
    def __init__(self, name, file_path, seperation_rate, is_normalized=False):
        
        self.name = name
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
        self.is_normalized = is_normalized
        
        print("DataGeneration object is created !!")
    
    # min, max 를 이용한 데이터 정규화
    def minmax_normalize(self, loaded_data):
        
        # 각 열의 최대값을 찾기 위해 행과 열을 바꾸어 줌. 즉 전치향렬을 만들어줌
        transpose_loaded_data = loaded_data.T        

        print("transpose_loaded_data.shape = ", transpose_loaded_data.shape)
        
        # 전치행렬을 위한 리스트
        transpose_normalize_data_list = []

        for index in range(len(transpose_loaded_data)):
            
            max_value = transpose_loaded_data[index].max()   # 각 행의 최대값을 찾음
            
            min_value = transpose_loaded_data[index].min()   # 각 행의 최소값을 찾음
            
            transpose_normalize_data_list.append( (transpose_loaded_data[index]-min_value) / (max_value-min_value) )
            
        # 리스트를 numpy type 으로 변환
        transpose_normalize_data = np.array(transpose_normalize_data_list)

        print(transpose_normalize_data.shape)

        # 데이터 저장을 위해 다시 전치행렬을 통해 행과 열을 바꿈
        normalize_data = transpose_normalize_data.T

        print(normalize_data.shape)

        # normalize 파일저장
        data_save_path = './Normalize_' + self.name + '_data.csv'
        
        np.savetxt(data_save_path, normalize_data, delimiter=',')
        
        return normalize_data

    # shuffle 기능을 이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기

        loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
        
        print("[debug. before data normalization] loaded_data[0] = ", loaded_data[0])

        print("loaded_data.shape = ", loaded_data.shape)
        
        if (self.is_normalized == True):
            
            loaded_data = self.minmax_normalize(loaded_data)
            
            print("[debug. after data normalization] loaded_data[0] = ", loaded_data[0])

        # 임시 저장 리스트
        training_data_list = []
        test_data_list = []

        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        print("total_data_num = ", total_data_num, ", test_data_num = ", test_data_num)

        # 전체 데이터 인덱스를 가지고 있는 리스트 생성
        total_data_index_list = [ index for index in range(total_data_num) ]

        # random.shuffle 을 이용하여 인덱스 리스트 생성
        random.shuffle(total_data_index_list)  # 전체 인덱스가 랜덤하게 섞여진 리스트로 변형된다

        # test data 를 위한 인덱스는 total_data_index_list 로뷰터 앞에서 40 % 의 데이터 인덱스
        test_data_index_list = total_data_index_list[ 0:test_data_num ]

        print("length of test_data_index_list = ", len(test_data_index_list))

        # training data 를 위한 인덱스는 total_data_index_list 에서 test data 인덱스를 제외한 나머지 부분
        training_data_index_list = total_data_index_list[ test_data_num: ]

        print("length of training_data_index_list = ", len(training_data_index_list))

        # training data 구성
        for training_data_index in training_data_index_list:
    
            training_data_list.append(loaded_data[training_data_index])

        # test data 구성
        for test_data_index in test_data_index_list:
    
            test_data_list.append(loaded_data[test_data_index])

        # generate training data from training_data_list using np.arrya(...)
        training_data = np.array(training_data_list)

        # generate test data from test_data_list using np.arrya(...)
        test_data = np.array(test_data_list)

        # verification shape
        print("training_data.shape = ", training_data.shape)
        print("test_data.shape = ", test_data.shape)

        # save training & test data (.csv)
        training_data_save_path = './random_' + self.name + '_training_data.csv'
        test_data_save_path = './random_' + self.name + '_test_data.csv'
        
        np.savetxt(training_data_save_path, training_data, delimiter=',')
        np.savetxt(test_data_save_path, test_data, delimiter=',')
        
        return training_data, test_data

In [3]:
# DataGeneration 객체 생성
seperation_rate = 0.3
data_obj = DataGeneration('ThoracicSurgery', './ThoracicSurgery.csv', seperation_rate, True)  # normalize 

# training_data, test_data 생성
(training_data, test_data) = data_obj.generate()

print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

DataGeneration object is created !!
[debug. before data normalization] loaded_data[0] =  [293.    1.    3.8   2.8   0.    0.    0.    0.    0.    0.   12.    0.
   0.    0.    1.    0.   62.    0. ]
loaded_data.shape =  (470, 18)
transpose_loaded_data.shape =  (18, 470)
(18, 470)
(470, 18)
[debug. after data normalization] loaded_data[0] =  [0.6226013  0.         0.4855967  0.02156081 0.         0.
 0.         0.         0.         0.         0.33333334 0.
 0.         0.         1.         0.         0.6212121  0.        ]
total_data_num =  470 , test_data_num =  141
length of test_data_index_list =  141
length of training_data_index_list =  329
training_data.shape =  (329, 18)
test_data.shape =  (141, 18)
training_data.shape =  (329, 18)
test_data.shape =  (141, 18)


In [4]:
training_x_data = training_data[ :, 0:-1]
training_t_data = training_data[ :, [-1]]

print("training_x_data.shape = ", training_x_data.shape)
print("training_t_data.shape = ", training_t_data.shape)

test_x_data = test_data[ :, 0:-1]
test_t_data = test_data[ :, [-1]]

print("test_x_data.shape = ", test_x_data.shape)
print("test_x_data.shape = ", test_x_data.shape)

training_x_data.shape =  (329, 17)
training_t_data.shape =  (329, 1)
test_x_data.shape =  (141, 17)
test_x_data.shape =  (141, 17)


In [5]:
# Hyper-Parameter Definition
learning_rate = 1e-3
input_nodes = training_x_data.shape[1]  # 17
hidden_nodes = 10
output_nodes = 1
epochs = 1

In [6]:
# 입력과 출력을 위한 플레이스홀더 정의
X_DATA = tf.placeholder(tf.float32, [None, input_nodes])  
T_DATA = tf.placeholder(tf.float32, [None, output_nodes])  

W2 = tf.Variable(tf.random_normal([input_nodes, hidden_nodes]))  # 은닉층 가중치 노드
b2 = tf.Variable(tf.random_normal([hidden_nodes]))               # 은닉층 바이어스 노드

W3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes])) # 출력층 가중치 노드
b3 = tf.Variable(tf.random_normal([output_nodes]))               # 출력층 바이어스 노드

In [7]:
# feed forward
Z2 = tf.matmul(X_DATA, W2) + b2
A2 = tf.sigmoid(Z2)

Z3 = tf.matmul(A2, W3) + b3  
y = A3 = tf.sigmoid(Z3)    # 최종 계산값

# 손실함수는 Cross-Entropy 
loss = -tf.reduce_mean( T_DATA*tf.log(y) + (1-T_DATA)*tf.log(1-y) ) 

In [8]:
# GradientDescent 
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [9]:
# 정확성 검사, True if y > 0.5 else False

# y shape 은 (759X8) (8X1) = 759X1, 즉 759개의 모든 데이터에 대해 y > 0.5 검사함
# 그러므로 predicted 또한 759 개의 1 또는 0 을 가짐
predicted = tf.cast(y > 0.5, dtype=tf.float32)  


# predicted 와 T 같으면 True 를 리턴하므로 cast 에 의해서 1로 강제 변환, 그렇지 않으면 0으로 변환해서
# accuracy 는 총 759 개의 1 또는 0 을 가짐. 이것을 전체 갯수 759로 나눔
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T_DATA), dtype=tf.float32))

In [10]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화

    start_time = datetime.now()
    
    for i in range(epochs):
    
        for step in range(10001):
      
            loss_val, _ = sess.run([loss, train], feed_dict={X_DATA: training_x_data, T_DATA: training_t_data})    
        
            if step % 500 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now()
    
    print("")
    print("Elapsed Time => ", end_time-start_time)
    
    # Accuracy 확인
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict={X_DATA: test_x_data, T_DATA: test_t_data})
    
    print("\ny_val.shape = ", y_val.shape, ", predicted_val = ", predicted_val.shape)    
    print("\nAccuracy = ", accuracy_val)

epochs =  0 , step =  0 , loss_val =  0.4381559
epochs =  0 , step =  500 , loss_val =  0.4349447
epochs =  0 , step =  1000 , loss_val =  0.43318954
epochs =  0 , step =  1500 , loss_val =  0.43212998
epochs =  0 , step =  2000 , loss_val =  0.43141523
epochs =  0 , step =  2500 , loss_val =  0.4308767
epochs =  0 , step =  3000 , loss_val =  0.43043053
epochs =  0 , step =  3500 , loss_val =  0.43003452
epochs =  0 , step =  4000 , loss_val =  0.42966664
epochs =  0 , step =  4500 , loss_val =  0.42931545
epochs =  0 , step =  5000 , loss_val =  0.42897478
epochs =  0 , step =  5500 , loss_val =  0.42864156
epochs =  0 , step =  6000 , loss_val =  0.42831415
epochs =  0 , step =  6500 , loss_val =  0.42799136
epochs =  0 , step =  7000 , loss_val =  0.42767283
epochs =  0 , step =  7500 , loss_val =  0.42735776
epochs =  0 , step =  8000 , loss_val =  0.42704606
epochs =  0 , step =  8500 , loss_val =  0.4267379
epochs =  0 , step =  9000 , loss_val =  0.42643353
epochs =  0 , step =